In [81]:
import psycopg2
import pandas as pd

In [ ]:
def validate_user(login, password):
    return 1

`docker run --name caremore -h caremore -p 5432:5432 -itd -e POSTGRES_PASSWORD=postgres -e POSTGRES_USER=caremore -e POSTGRES_DB=caremore postgres:10`

```sql
INSERT INTO public."Customer"(
	"CustomerID", "FirstName", "LastName", "Birthdate", "Gender", "CustomerLogin", "CustomerPassword")
	VALUES (1, 'Goke', 'Alabi', '1990-01-01', 'Male', 'alabi', 'goke');

INSERT INTO public."HealthcareCenter"(
	"ID", "Name", "Address", "Latitude", "Longitude", "PostalCode", "ContactPhone", "ContactEmail")
	VALUES (1, 'Crags Healthcare Center', '10 Bowmont Pl Edinburgh', 55.944772, -3.179681, 'EH8 9RY', '440750000000', 'healthcare@crags.org');

INSERT INTO public."Appointment"(
	"ID", "CustomerID", "HealthcareCenterID", "Time", "Date")
	VALUES (1, 1, 1, '04:15 PM', '2020-02-24');

INSERT INTO public."Perk_Assignment"(
	"PerkID", "CustomerID", "AddedDate", "PerkAssignmentID")
	VALUES (1, 1, '2020-02-22', 1);

INSERT INTO public."Perk"(
	"PerkID", "Name", "PerkType", "PartnerID", "Quantity", "Units")
	VALUES (1, 'Sausage Roll', 'Food', 1, 1, 1);

INSERT INTO public."Partner"(
	"PartnerID", "Name", "Address", "PostalCode")
	VALUES (1, 'Greggs', 'Edinburgh', 'EH8 9PP');


```

In [82]:
try:
    conn = psycopg2.connect("dbname='caremore' user='caremore' host='localhost' password='postgres'")
except:
    print("I am unable to connect to the database")

In [83]:
def get_next_appointment(customer_id):
    sql = '''select c."FirstName"
            , c."LastName"
            , h."Address"
            , a."Date"
            , a."Time"
    from public."Appointment" as a inner join public."Customer" as c
            on a."CustomerID" = c."CustomerID"
            inner join public."HealthcareCenter" as h
            on a."HealthcareCenterID" = h."ID"
    where c."CustomerID" = {};'''.format(customer_id)
    df = pd.read_sql(sql, conn)
    return df

def get_perk_for_customer(customer_id):
    sql = '''select c."FirstName"
            , c."LastName"
            , p."Name" as perk
            , pa."ValidUntil"
            , pt."Name" as partner
            --, pa.* 
            --, p.*
            --, c.*
            from public."Perk_Assignment" as pa inner join public."Perk" as p
                on pa."PerkID" = p."PerkID"
                inner join public."Customer" as c
                on pa."CustomerID" = c."CustomerID"
                inner join public."Partner" as pt
                on p."PartnerID" = pt."PartnerID"
            where c."CustomerID" = {};'''.format(customer_id)
    df = pd.read_sql(sql, conn)
    return df

In [84]:
login = input('Welcome, what is your login?: ');
print('Welcome {}'.format(login))

customer_id = validate_user(login, "*****")
next_appointment = get_next_appointment(customer_id)
perk = get_perk_for_customer(customer_id)

message_appointment = '''Hi {}
Your Next Appointment
GP Appointment
{} 
{}
{}'''.format(next_appointment.iloc[0].FirstName
                       , next_appointment.iloc[0].Date
                       , next_appointment.iloc[0].Time
                       , next_appointment.iloc[0].Address)

message_perk = '''This week`s bonus
{} from {}'''.format(perk.iloc[0].perk
                    , perk.iloc[0].partner)

Welcome, what is your login?: galadi
Welcome galadi


In [85]:
print(message_appointment)
print()
print(message_perk)

Hi Goke
Your Next Appointment
GP Appointment
2020-02-24 
04:15 PM
10 Bowmont Pl Edinburgh

This week`s bonus
Sausage Roll from Greggs


In [86]:
message_appointment

'Hi Goke\nYour Next Appointment\nGP Appointment\n2020-02-24 \n04:15 PM\n10 Bowmont Pl Edinburgh'

In [87]:
perk

,FirstName,LastName,perk,ValidUntil,partner
0,Goke,Alabi,Sausage Roll,2020-03-01,Greggs
